In [ ]:
# Common imports
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import nengo
import nengo.utils.numpy as npext
import nengo_ocl
import nengo_gui.ipython

import phd

# Some plotting niceties
plt.rc('figure', figsize=(10, 8))
sns.set_style('white')
sns.set_style('ticks')

In [ ]:
fs = 25000.
dt = 1. / fs

def plot_sound(process, t, dt):
    plt.figure()
    plt.plot(process.trange(t, dt=dt), process.run(t, dt=dt))
    plt.xlim(right=t)
    sns.despine()

plot_sound(phd.sounds.WavFile('speech.wav'), 0.667, dt)
plot_sound(phd.sounds.WhiteNoise(), 0.1, dt)
plot_sound(phd.sounds.Tone(250), 0.1, dt)

#  Recognition system

## Auditory periphery

Making heavy use of [Brian hears](http://www.briansimulator.org/docs/hears.html),
but should also investigate other periphery models.

In [ ]:
fs = 20000.
freqs = phd.filters.erbspace(20, 10000, 64)
sound = phd.sounds.WhiteNoise()
aud_filter = phd.filters.dual_resonance(freqs)

model = phd.SpeechRecognition()
model.add_periphery(freqs, sound, aud_filter, fs=fs)
ihc_p, an_in_p, an_p = model.probe_periphery()

In [ ]:
from nengo.utils.matplotlib import rasterplot

dt = 1. / freqs.max()
print("dt=%.5f" % dt)
sim = nengo.Simulator(model, dt=dt*.5)
sim.run(0.1)

plt.figure()
phd.plots.cochleogram(sim.data[ihc_p], freqs, sim.trange())
plt.figure()
phd.plots.cochleogram(sim.data[an_in_p], freqs, sim.trange())
plt.figure()
rasterplot(sim.trange(), sim.data[an_p])
plt.ylim(0, model.an.n_neurons * model.an.n_ensembles)

In [ ]:
print(sum(ens.n_neurons for ens in model.all_ensembles))

## Preprocessing layer

In [ ]:
fs = 20000.
freqs = phd.filters.erbspace(20, 10000, 64)
sound = phd.sounds.WavFile('speech.wav')
aud_filter = phd.filters.dual_resonance(freqs)

model = phd.SpeechRecognition()
model.add_periphery(freqs, sound, aud_filter, fs=fs)
# TODO: Vary tau_highpass and radius
model.add_derivative(n_neurons=30, delay=0.01, tau_highpass=0.05)
with model:
    ihc_p = nengo.Probe(model.ihc, synapse=None)
    an_p = nengo.Probe(model.an.output, synapse=0.01)
d_p = model.probe_derivative(delay=0.01)

In [ ]:
dt = 1. / freqs.max()
sim = nengo.Simulator(model, dt=dt*.5)
sim.run(0.667)

plt.figure()
phd.plots.cochleogram(sim.data[ihc_p], freqs, sim.trange())
plt.figure()
phd.plots.cochleogram(sim.data[an_p], freqs, sim.trange())
plt.figure()
phd.plots.cochleogram(sim.data[d_p], freqs, sim.trange())

In [ ]:
print(sum(ens.n_neurons for ens in model.all_ensembles))

## Feature layer